# Anomalia magnética

## Carregando os dados

In [ ]:
from __future__ import division, print_function
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
x, y, z, anom = np.loadtxt('anomalia-mag.txt', unpack=True)
shape = (50, 50)

In [ ]:
plt.tricontourf(y, x, anom, 60, cmap='RdBu_r')
plt.colorbar()

## Calcula o TGA

In [ ]:
from fatiando.gravmag import transform

In [ ]:
tga = transform.tga(x, y, anom, shape)

In [ ]:
plt.tricontourf(y, x, tga, 60, cmap='RdBu_r')
plt.colorbar()

In [ ]:
transform.

## Desafio: Calcule a derivada vertical e o gradiente total horizontal

In [ ]:
dx = transform.derivx(x, y, anom, shape)
dy = transform.derivy(x, y, anom, shape)
dz = transform.derivz(x, y, anom, shape)
thd = np.sqrt(dx**2 + dy**2)

In [ ]:
plt.tricontourf(y, x, thd, 60, cmap='RdBu_r')
plt.colorbar()

In [ ]:
plt.tricontourf(y, x, dz, 60, cmap='RdBu_r')
plt.colorbar()

## Localizando a fonte com deconvolução de Euler

In [ ]:
from fatiando.gravmag import euler

In [ ]:
solver = euler.Classic(x, y, z, anom, dx, dy, dz, structural_index=3)

In [ ]:
solver.estimate_

In [ ]:
solver.fit()

In [ ]:
print(solver.estimate_)

In [ ]:
plt.tricontourf(y, x, anom, 60, cmap='RdBu_r')
plt.colorbar()
plt.plot(solver.estimate_[1], solver.estimate_[0], 'ok')

## Desafio: Varie o indice estrutural 

Rode com os indices `[1, 2, 3]` e faça um gráfico de índice pelo z estimado.

In [ ]:
zs = []
indices = [1, 2, 3]
for index in indices:
    solver = euler.Classic(x, y, z, anom, dx, dy, dz, index).fit()
    zs.append(solver.estimate_[2])

In [ ]:
plt.plot(indices, zs, 'o-k')

## Várias soluções usando janelas

In [ ]:
solver = euler.Classic(x, y, z, anom, dx, dy, dz, 3)
janela = euler.MovingWindow(solver, (10, 10), (3000, 3000))
janela.fit()

In [ ]:
janela.estimate_

In [ ]:
plt.tricontourf(y, x, anom, 60, cmap='RdBu_r')
plt.colorbar()
plt.plot(janela.estimate_[:, 1], janela.estimate_[:, 0], 'ok')

In [ ]:
plt.tricontourf(y, x, anom, 60, cmap='RdBu_r')
plt.scatter(janela.estimate_[:, 1], janela.estimate_[:, 0], s=100, c=janela.estimate_[:, 2], cmap="Greens")
plt.colorbar()

## Desafio: Faça um plot 3D das solução de Euler

In [ ]:
from fatiando.vis import myv

In [ ]:
myv.figure()
myv.points(janela.estimate_, size=50, color=(1, 0, 0))
bounds = [0, 8000, 0, 10000, 0, 1000]
myv.axes(myv.outline(bounds))
myv.wall_north(bounds)
myv.wall_bottom(bounds)
myv.show()

## Desafio: Faça a  redução ao polo

In [ ]:
inc, dec = -19.5, -18.5

In [ ]:
polo = transform.reduce_to_pole(x, y, anom, shape, inc, dec, inc, dec)

In [ ]:
plt.tricontourf(y, x, polo, 60, cmap='RdBu_r')
plt.colorbar()

## Desafio: Estime a direção de magnetização e reduza ao polo de novo

In [ ]:
from fatiando.gravmag import magdir

In [ ]:
solver = euler.Classic(x, y, z, anom, dx, dy, dz, structural_index=3).fit()

In [ ]:
direcao = magdir.DipoleMagDir(x, y, z, anom, inc, dec, [solver.estimate_])

In [ ]:
direcao.fit()

In [ ]:
direcao.estimate_

In [ ]:
sinc, sdec = direcao.estimate_[0][1:]

In [ ]:
polo = transform.reduce_to_pole(x, y, anom, shape, inc, dec, sinc, sdec)

In [ ]:
plt.tricontourf(y, x, polo, 60, cmap='RdBu_r')
plt.colorbar()